# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [2]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [3]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [4]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [5]:
prompt = """
Dime quiénes ganaron los títulos individuales masculino y femenino en Wimbledon 2023.  Incluye sus nombres completos si es posible.
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: Lo siento, como modelo de lenguaje de IA, no tengo la capacidad de predecir eventos futuros con precisión. Mi función es proporcionar respuestas basadas en datos y conocimientos existentes. Por lo tanto, no puedo responder a esta pregunta. Por favor, consulte los resultados de Wimbledon 2023 después de que se hayan jugado los partidos.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [6]:
text = f""" 
Hace año los Premios Nobel de Física y Química fueron entregados a personas que están apoyando el ecosistema de inteligencia artificial, como lo es Joe Hilton o the Miss, hashtag del equipo de Deadmin. Y esto es bastante controversial. Muchas personas dentro de la Comunidad de los Premios Nobel y de Investigación hablan de El premio de física debe ser entregado a físicos que están trabajando en distintos proyectos y no hacia la creación de inteligencia artificial. Por otro lado, también pasa lo mismo con el premio de química, pero es que la verdad es que está bastante interesante y no, esto no tiene nada que ver con chat GMT, con Dalí, con ninguno de estos modelos que están siendo truena. Tendencia de inteligencia artificial degenerativa, sino con esfuerzos de décadas en la construcción de grandes modelos de inteligencia artificial que están apoyando, sí, a la física, sí a la química, pero como lo vemos también al impacto. En salud, en educación, en economía. En paz, en comunicaciones y distintos usos increíbles que se le está dando a la inteligencia artificial. Y justo vamos a hablar de eso el día de hoy. Vamos a empezar con el premio de física fue entregado a John Hoffman y a Jofrey Hilton. Y esto es por sus esfuerzos en distintas investigaciones para poder llegar a lo que hoy conocemos como las redes neuronales, que es una base del Deep learning y esta es la base de todo lo que conocemos a hoy como inteligencia artificial, generativa, Deep learning y todas las maravillas que estamos viendo de no ser por esas investigaciones. No existirían las redes neuronales artificiales como las conocemos y, por supuesto, no estaría generándose este gran impacto en inteligencia artificial que vivimos. ¿A hoy en día ahora, por qué específicamente se le dieron estos premios a estas personas? Empecemos por John Hotfield, y es que en 1982 se creó un gran diseño para lo que conocemos como memoria asociativa, y es que cabe resaltar que a hoy, la red neuronales artificiales que utilizamos en inteligencia artificial están inspiradas en las redes neuronales biológicas. Que habitan en nuestro cerebro, tal cual son distintas conexiones que se comunican y envían estímulos. Tras sí, para lograr lo que conocemos como pensamiento, ese increíble número de conexiones entre las distintas redes es lo que se quiso llevar hacia las redes neuronales artificiales, que funcionan de manera muy similar. Y ahora, en la investigación de 1982 de Hotfield, lo que se quería era buscar una memoria, algo que pudiera identificar patrones. Y llegar a cierto punto en la red neuronal para que se pueda. Generar o pueda encontrar este patrón que funciona exactamente cómo funciona nuestra memoria, supongamos esto. Muchas veces, cuando olvidas un término, intentas buscar patrones o cosas similares que puedan ayudar a recordarlo. Y pasa mucho en conversaciones tipo esta persona que empezaba por F. Francisco Felipe, ese tipo de cosas o el que ha actuado en esta película junto a Brad Pitt y George Clooney y busca ciertos patrones o asociaciones que te ayuden. A llegar a ese término en este Premio Nobel se ilustra bastante bien esta investigación. Con esta imagen o esta ilustración. Imaginemos un Valle, el cual tiene montañas y por supuesto tiene valles en la parte de abajo o en los valles. Esta es realmente la memoria profunda, los conceptos. Más sólidos que tiene una red neuronal, supongamos que arrojamos una esfera en algún punto de la montaña que estas montañas van a hacer una similitud a los patrones. Esta canica o el objeto que empezamos a robar a través de esta montaña, va a buscar y va a caminar entre los patrones hasta que llegue en algún punto al punto más bajo de todo ese Valle, que es simplemente un punto donde se encuentra una memoria. Si la red neuronal está bien construida, podemos darle una imagen o una información incompleta o difusa y permitir que el modelo. De inteligencia artificial o de memoria asociativa. Propuesto por headfeld, logre completarlo hasta llegar a la memoria y completar totalmente este input que fue difuso. Es un poco complejo, es un poco complicado, pero realmente es una de las bases que nos ayudó a llegar a lo que conocemos hoy como Deep learning o redes neuronales artificiales bastante robustas. Por otro lado, también del premio de física está Geoffrey Hilton, 1 de los padrinos de la inteligencia artificial que ha hecho. Increíbles aportes en estadística y física totalmente orientada a las redes. Estoy hablando de que continuamente está entregando nuevos papers, nuevas investigación que permita seguir adelante la inteligencia artificial a hoy, curiosamente, son las personas que más se está apoyando la regulación de inteligencia artificial, incluso muchas veces parar la investigación en inteligencia artificial por ciertos incertidumbres o por ciertas miedos que tiene. ¿Eso, Jeffrey Hilton, sobre cómo se van a utilizar? Toda su investigación y las redes neuronales y la inteligencia artificial a hoy en el modelo económico que tenemos, con bastantes miedos y él está insistiendo en una regulación mucho más fuerte. Pero esto es tema de otro video. Lo interesante es que Jofrey Hilton nos ayudó inicialmente en el 87 a crear la máquina de boltzmann, y esto es basado, por supuesto, también en una investigación física. El objetivo era entregar probabilidades a ciertos patrones de la red con el fin de que aprendiera por sí mismo. Y es que, tal cual como funcionan las redes neuronales biológicas, la idea es que una red de inteligencia artificial o una red neuronal aprenda por sí misma. Cuando le doy distintos ejemplos, pasa tal cual como con nuestro cerebro. Si le digo esto es un perro y esto es un gato, a un niño o a 1 bebé. Y se lo digo continuamente, su cerebro va a empezar a interpretar los patrones del perro. Que son diferentes a los patrones del gato. En cuanto a su forma en cuanto a los sonidos que emiten. Y también empieza a relacionar el concepto perro y el concepto gato en su lenguaje, para así entender realmente o aprender qué son perros y gatos. Y eso se logra en inteligencia artificial. Gracias a este avance, Joffrey Hilton nos entrega también un concepto interesante que son las capas ocultas de una red neuronal. En verdad, las capas ocultas empiezan a guardar. Información o patrones más relevantes entre más profunda sea una red de inteligencia artificial, una red neuronal. Tiene más capas ocultas y estas capas ocultas conllevan información mucho más densa y específica al punto en el que no sabemos qué es lo que guarda o no sabemos qué patrones fue detectar, pero sí que está aprendiendo a reconocer a y a identificar distintos patrones. Y claro, llegamos a este punto en donde muchas personas están hablando de hey, si bien hace décadas se hizo esta implementación que si bien está basado en física, es muy interesante. No es suficiente para poder ganar un premio Nobel en física, y menos aún cuando hay grandes investigaciones y grandes personas trabajando en proyectos interesantísimos de física actualmente y no estos proyectos que ayudan a llegar a lo que hoy conocemos como inteligencia artificial de learning e inteligencia artificial generativa. Por otro lado, las personas del Premio Nobel y muchos. Importantes científicos hablan de la relevancia de que esas investigaciones en verdad tienen una base fundamental de la física y la estadística. Que esto, por supuesto, está ayudando a físicos alrededor del mundo y a muchos investigadores a mejorar los resultados de sus proyectos o su investigación. De hecho, gracias a esto se crearon redes neuronales como las conocemos actualmente. Y esas redes neuronales se están utilizando, por ejemplo, para analizar datos en el CERN para el bosón de Higgs. Se están utilizando incluso para poder procesar. La foto del primer agujero ***** que vemos anteriormente. Se están utilizando también para cruzar distintos datos y poder generar fotos del espacio con telescopios de altísimo nivel como los conocemos actualmente. Los usos de esta tecnología orientada física también existen. No solo tienen un fundamento físico, sino que también se está utilizando para mejorar la investigación de la física. Y esto, por supuesto, es lo que dice el equipo de Premios Nobel. Se hace una discusión muy fuerte. Algunas personas están a favor, otras personas están en contra, así que tu opinión es importante. Si quieres, déjala acá en el sistema de comentarios sobre si es suficiente estos aportes de inteligencia artificial para ganar un premio Nobel, y más en un área tan importante como lo en la física. Y Por otro lado, claro, tenemos el previo de química en los Nobel entregado al equipo. De Alpha Fold Athemes a Sabes que es el CEO y fundador de Deep mind. Duela laboratorios de inteligencia artificial más importantes que tenemos en El Mundo. Propiedad, por supuesto de Google o alphabet y que ha hecho grandes contribuciones. Claro, lo conocemos por germinar y lo conocemos por imagen, los conocemos por notebook LM, muchísimos modelos interesantes que por supuesto, hemos visto en este canal, que si no te has suscrito, pues sí, vale la pena que lo hagas porque me ayuda bastante a seguir generando este tipo de contenidos. Pero volviendo al tema de El Premio Nobel de Química. Esto es interesante porque se dio el Premio Nobel de Química gracias a un modelo específico de inteligencia artificial que, por supuesto, es el resultado de mucha investigación de químicos, de estadísticos, de físicos y de muchas personas que trabajan dentro del ecosistema de inteligencia artificial en deathman. Y estoy hablando de Alpha Fald y claro. No voy a entrar mucho en detalle de Alfa Foll porque ya dediqué un video entero para hablar de la importancia de este modelo. Lo puedes encontrar acá y lo puedes encontrar claro en la descripción de este vídeo en los enlaces lo importante es. Es un modelo de inteligencia artificial encargado de predecir la estructura. De las proteínas. Simplemente eso, pues, resulta que las proteínas y otras enzimas son la base de la biología, o sea, la base misma de la vida. Si logramos entender cómo funciona una proteína, o cuál es su estructura, o cuál es su diseño, logramos entender cómo funciona tal cual esa proteína. Y, por supuesto, podemos modificarla. ¿Podemos optimizarla? Podemos hacer muchísimas otras cosas. De hecho, la gran mayoría de proteínas que conocemos como humanidad son resultado de Alfa fold de un modelo de inteligencia artificial que ayuda a científicos alrededor del mundo a descubrir estas proteínas que son de nuevo la base de la cadena de la vida. ¿Cuando logramos descubrir eso, entonces podemos mejorar los medicamentos y las vacunas? Podemos crear enzimas que descompongan el plástico o que encapsulen el C o 2. Que por supuesto esto. Muchísimo. El impacto ambiental, podemos prevenir enfermedades, podemos detectar enfermedades de manera más temprana. Estamos hablando de un modelo que literalmente nos está ayudando a decodificar la clave de la vida. Y por supuesto, esto fue suficiente para que el equipo de los Premios Nobel le diera este reconocimiento al equipo de Deadmin y a Alpha Fold de nuevo. Si quieres saber mucho más de este modelo, lo encuentras por acá. O en el enlace que está en la descripción de este vídeo. Quiero terminar este vídeo con una reflexión y es que claro, en este canal hablo muchísimo de. Chatbots de modelos que pueden generar imágenes de manejo de video, manejo de audio de distintas arquitecturas que están ayudando al entorno empresarial y económico. De cómo incluso podemos utilizar inteligencia artificial para hacer diseño, para mejorar en programación, para mejorar en distintos aspectos, incluso en nuestro día a día. Pero esto es lo que lo utilizamos nosotros directamente, como clientes finales o como customers. En realidad, la inteligencia artificial va mucho más allá de chat. GPT va mucho más allá de Proms. Va mucho más allá de inteligencia artificial generativa que es esta ola que estamos viviendo. Inteligencia artificial está ayudando a resolver. Grandes problemas que afrontamos en la vida, como lo es el manejo de proteínas, como lo vimos en el premio de química, como lo es la investigación espacial, que está apoyando esa inteligencia artificial para ayudar a la investigación de físicos y matemáticos alrededor del mundo. Está sirviendo literalmente para mejorar los modelos de educación. Está sirviendo para mejorar la salud y los esquemas y las vacunas y todo lo que tenemos alrededor del mundo. Inteligencia artificial no es solo una moda. Venimos trabajándola desde 1950. En 1960 se creó el primer modelo de redes neuronales con un perceptrón en el 82. Hotfield nos entrega esta red de memoria asociativa en el 87. Estamos hablando de los esfuerzos de Jofrey Hilton. Lo que vivimos a hoy es un esfuerzo acumulado de estos y muchísimos otros investigadores que están ayudando. A crear herramientas y modelos de inteligencia artificial que mejoren y aceleren el estado actual que tenemos en investigación y en tecnología y en avance como humanidad. Así que vale mucho la pena que le demos un ojo a eso y entendamos el alcance real de la inteligencia artificial. Y claro, algunas personas hablarán de sí que está interesante. ¿No es para un Nobel? Quizás sí, quizás no puedes dejar tus comentarios. Por acá y no siendo más, nos vemos en otro vídeo, recuerda. ¿Qué si te ha gustado esto? Suscríbete activa la campanita, dale like. Por supuesto, comparte este vídeo para que muchas personas más entiendan el verdadero impacto de inteligencia artificial."""

prompt = f"""
Actue como experto en resumen y extraccion de palabras claves y Quiero que me explique quieren ganaron el premio nobel de fisca en el año 2024 y porque en maxio 30 palabras.
```{text}```
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: John Hoffman y Jofrey Hilton ganaron el Premio Nobel de Física en 2024 por sus investigaciones en redes neuronales, base del Deep learning y la inteligencia artificial. Alpha Fold, modelo de inteligencia artificial encargado de predecir la estructura de proteínas, ganó el Premio Nobel de Química. Ambos premios son controversiales, pero la inteligencia artificial está ayudando a resolver grandes problemas en la vida, como la investigación espacial, la mejora de la salud y la educación.



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
